In [3]:
import System.Random

In [4]:
:set -XFlexibleContexts

In [5]:
g <- getStdGen
print . take 10 $ (randomRs ('a', 'z') g)


"rhsdsrqhog"

# Problem 21

In [36]:
insertAt a l c = insertAt a [] l c
    where
        insertAt a p [] c = p ++ [a]
        insertAt a p (x:sx) c
            | c == 1 = (p ++ [a] ++ x:sx)
            | c > 1 = insertAt a (p ++ [x]) sx (c-1)

Line 1: Eta reduce
Found:
insertAt a l c = insertAt a [] l c
Why not:
insertAt a = insertAt a []Line 5: Redundant bracket
Found:
(p ++ [a] ++ x : sx)
Why not:
p ++ [a] ++ x : sx

In [37]:
insertAt 'X' "abcd" 2

"aXbcd"

# Problem 22

In [38]:
range a b = [a .. b]

In [39]:
range 1 10

[1,2,3,4,5,6,7,8,9,10]

In [42]:
range :: Int -> Int -> [Int]

range a b
    | a == b = [b]
    | a < b = a:range (a+1) b

In [49]:
range 4 9

[4,5,6,7,8,9]

# Problem 23

In [94]:
rnd_select :: [Int] -> [a] -> [a]
randomElements a b = take a $ randomRs (0, length b - 1) g

rnd_select a l = rnd_select a l []
    where
        rnd_select (x:xs) l g
            | null xs = g ++ [l !! x]
            | otherwise = rnd_select xs l g ++ [l !! x]

rnd_select_ :: Int -> [a] -> [a]
rnd_select_ a l = rnd_select (randomElements a l) l

Line 4: Use camelCase
Found:
rnd_select a l = ...
Why not:
rndSelect a l = ...Line 11: Use camelCase
Found:
rnd_select_ a l = ...
Why not:
rndSelect_ a l = ...

In [95]:
rnd_select_ 3 "abcdefgh"

"hbc"

In [96]:
rnd_select a b = take a [b !! x | x <- randomRs (0, length b - 1) g]

Line 1: Use camelCase
Found:
rnd_select a b = ...
Why not:
rndSelect a b = ...

In [97]:
rnd_select_ 3 "abcdefgh"

"hbc"

# Problem 24

In [115]:
diff_select a b = take a [x | x <- randomRs (1, length b) g]

Line 1: Use camelCase
Found:
diff_select a b = ...
Why not:
diffSelect a b = ...Line 1: Redundant list comprehension
Found:
[x | x <- randomRs (1, length b) g]
Why not:
(randomRs (1, length b) g)

In [101]:
diff_select 6 49

[46,16,37,7,33,12]

# Problem 25

In [6]:
takeAfter a [] = []
takeAfter 0 xs = xs
takeAfter n (x:xs) = takeAfter (n-1) xs

removeItem n x = take (n-1) x ++ takeAfter n x

random_perm :: [a] -> [a]
random_perm l = random_perm l []
    where 
        random_perm l b
            | length l == 1 = b ++ l
            | otherwise = random_perm (removeItem x l) ((l !! (x - 1)):b)
            where
                x = head (randomRs (1, length l) g)

Line 8: Use camelCase
Found:
random_perm l = ...
Why not:
randomPerm l = ...

In [7]:
random_perm "abcdef"

"edfcba"

# Problem 26 (ugly) :c

In [67]:
les (Just a) (Just b)
    | (a < b) = Just b
    | otherwise = Nothing
les Nothing (Just _) = Nothing

isp (Just _) = True
isp (Nothing) = False

msk (x:l) (y:b) = msk (x:l) (y:b) []
    where
        msk [] [] g = reverse g
        msk (x:l) (y:b) g
            | y = msk l b (x:g)
            | otherwise = msk l b g
            
combinations n l =
    let tuples = sequence (replicate n l)
        masking = map isp $ map (\i -> foldl1 les (Just <$> i)) tuples
    in msk tuples masking

Line 2: Redundant bracket
Found:
(a < b)
Why not:
a < bLine 7: Redundant bracket
Found:
(Nothing)
Why not:
NothingLine 17: Use replicateM
Found:
sequence (replicate n l)
Why not:
Control.Monad.replicateM n lLine 18: Use map once
Found:
map isp $ map (\ i -> foldl1 les (Just <$> i)) tuples
Why not:
map (isp . (\ i -> foldl1 les (Just <$> i))) tuples

In [74]:
combinations 3 [1,2,3,4]

[[1,2,3],[1,2,4],[1,3,4],[2,3,4]]

In [75]:
combinations 3 "abcdef"

["abc","abd","abe","abf","acd","ace","acf","ade","adf","aef","bcd","bce","bcf","bde","bdf","bef","cde","cdf","cef","def"]

# Problem 27

In [145]:
permutate [] = [[]]
permutate l = [a:x | a <- l, x <- permutate $ filter (/= a) l]

In [150]:
group_ [] _ = []
group_ (a:g) l = [take a l] ++ group_ g (drop a l)
group g = (map (group_  g)) . permutate

Line 2: Use :
Found:
[take a l] ++ group_ g (drop a l)
Why not:
take a l : group_ g (drop a l)Line 3: Redundant bracket
Found:
(map (group_ g)) . permutate
Why not:
map (group_ g) . permutate

In [151]:
group [1,2] [1,2,3]

[[[1],[2,3]],[[1],[3,2]],[[2],[1,3]],[[2],[3,1]],[[3],[1,2]],[[3],[2,1]]]

# Problem 28 (a)

In [174]:
lenless a b
    | length a == length b = a < b
    | otherwise = length a < length b

lenge a b
    | length a == length b = a >= b
    | otherwise = length a >= length b


lsort [] = []
lsort [x] = [x]
lsort (x:xs) = lsort (filter (lenge x) xs) ++ (x:lsort (filter (lenless x) xs))

In [175]:
lsort ["abc","de","fgh","de","ijkl","mn","o"]

["o","de","de","mn","abc","fgh","ijkl"]

# Problem 28 (b)

In [266]:
lens = map length

In [267]:
freq :: Int -> [Int] -> Int
freq n a = sum (map ((\i -> if i then 1 else 0) . (==n)) a)

In [268]:
mapFreq :: [[a]] -> [(Int, [a])]
mapFreq a = map (\i -> (freq (length i) (lens a), i)) a

In [269]:
lenge (a, c) (b, d)
    | a == b = c >= d
    | otherwise = a >= b
    
lenless (a, c) (b, d)
    | a == b = c < d
    | otherwise = a < b


lsort [] = []
lsort [x] = [x]
lsort (x:xs) = lsort (filter (lenge x) xs) ++ (x:lsort (filter (lenless x) xs))

In [273]:
lfsort = map snd . lsort . mapFreq

In [274]:
lfsort ["abc", "de", "fgh", "de", "ijkl", "mn", "o"]

["ijkl","o","abc","fgh","de","de","mn"]